<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

## Instalacja bibliotek

W celu instalacji niezbędnych bibliotek z ograniczoną ilością zwracanych informacji, można:
- zastosować flagę '-q' (quiet) dla przy każdej bibliotece, która sprawi, że wyświetli się mniej informacji
- użyć komendy %%capture na początku komórki, która przechwytuje wyniki wykonywania kodu w danej komórce

In [ ]:
!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install torch

In [ ]:
import torch
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

# Gated models
W przypadku modeli, na które zostały nałożone zabezpieczenia w postaci kontrolowanego dostępu (gated models), wymagane będzie podanie tokena użytkownika, dostępnego na portalu HuggingFace: <br>
- informacje dotyczące tokenów: <br>
https://huggingface.co/docs/hub/security-tokens <br>
- token użytkownika <br>
https://huggingface.co/settings/tokens

W tym celu należy zainstalować bibliotekę "huggingface_hub[cli]" oraz dokonać logowania za pomocą tokena, co zostało zaprezentowane w poniższych 2 komórkach.

In [ ]:
!pip install -U "huggingface_hub[cli]"

In [ ]:
!huggingface-cli login

### Inicjalizacja modelu (4 bit)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = 'speakleash/Bielik-11B-v2.2-Instruct'
load_4_bit = True

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = None

if load_4_bit:
    quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
    )

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config
                                             )

### Funkcja pomocnicza do generowania odpowiedzi

In [ ]:
import uuid
import json

In [ ]:
temperature = 0.7
max_tokens = 256
top_k = 100
top_p = 1


def generate(prompt, system=None):
    messages = []

    if system:
        messages.append({"role": "system", "content": system})

    messages.append({"role": "user", "content": prompt})

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = input_ids

    # Sprawdzenie dostępności CUDA i przeniesienie danych wejściowych na odpowiednie urządzenie (GPU lub CPU)
    if torch.cuda.is_available():
        model_inputs = input_ids.to(device)

    # generowanie odpowiedzi
    outputs = model.generate(model_inputs,
                             streamer=streamer,
                             max_new_tokens=max_tokens,
                             do_sample=True if temperature else False,
                             temperature=temperature,
                             top_k=top_k,
                             top_p=top_p)

    # zapis do pliku
    filename = f"output{str(uuid.uuid4())[:8]}.txt"

    with open(filename, "w", encoding="utf-8") as file:
        content = {
                "prompt": messages,
                "output": tokenizer.batch_decode(outputs, skip_special_tokens=False)
        }
        json.dump(content, file, ensure_ascii=False, indent=4)

#### Test

In [ ]:
generate("Hej, jesteś tam?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.


:) Jestem tutaj, żeby Ci pomóc. W czym mogę Ci dzisiaj pomóc?


In [ ]:
generate("Hej, czym się zajmujesz?", system = "Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Jestem programistą baz danych i zajmuję się głównie tworzeniem i utrzymywaniem relacyjnych baz danych SQL oraz NoSQL. Dodatkowo poszerzam swoją wiedzę w dziedzinie sztucznej inteligencji i uczę się o modelach uczenia maszynowego.

Moja praca obejmuje m.in.:
- Projektowanie i implementację schematów baz danych
- Pisanie zapytań SQL i NoSQL
- Optymalizację wydajności baz danych
- Tworzenie i utrzymywanie kopii zapasowych i archiwów danych

Aktualnie dużo czasu poświęcam na naukę:
- Algorytmów uczenia maszynowego
- Głębokiego uczenia
- Przetwarzana języka naturalnego (NLP)
- Przetwarzania obrazów i wideo

Ponadto interesuję


# Przykład 1

Po prostu **"Hello World"**, a bardziej **"Witaj świecie!"**

In [ ]:
temperature = 1.0
max_tokens = 256

prompt = "Witaj świecie!"
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Oto klasyczna formuła powitania z wykorzystaniem frazy "Witaj świecie!".

Czy chciałbyś, żebym wyjaśnił, co oznacza ta fraza?

### Wyjaśnienie:

Fraza "Witaj świecie!" to popularne powitanie używane w kontekście programowania, szczególnie w językach takich jak C, C++, Python czy Java. Gdy rozpoczynasz pracę z nowym programem lub plikiem z kodem, ten krótki, ale znaczący napis "Witaj świecie!" przywołuje uśmiech na twarz i przypomina, że program został pomyślnie uruchomiony.

Kiedy pracując nad programem widzisz na ekranie swój ukochany napis "Witaj świecie!", to zazwyczaj znaczy, że Twoja praca posuwa się do przod


# Przykład 2

Zadawanie pytań otwartych

In [ ]:
temperature = 0.1
max_tokens = 512

prompt = "Co to jest fotosynteza?"
generate(prompt)

In [ ]:
temperature = 0.1
max_tokens = 512

prompt = "Co to jest fotosynteza?"
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Fotosynteza to proces, w którym rośliny, algi i niektóre bakterie przekształcają energię świetlną w energię chemiczną, magazynując ją w postaci związków organicznych, głównie glukozy. Proces ten zachodzi w chloroplastach, a dokładniej w tylakoidach chloroplastów, gdzie znajduje się zielony barwnik – chlorofil.

Fotosynteza składa się z dwóch głównych etapów:

1. Faza jasna (reakcje świetlne):
  - Zachodzi w błonach tylakoidów.
  - Energia świetlna jest pochłaniana przez chlorofil i inne barwniki.
  - Energia ta jest wykorzystywana do rozszczepienia cząsteczek wody (H₂O) na tlen (O₂), protony (H⁺) i elektrony (e⁻).
  - Tlen jest uwalniany do atmosfery jako produkt uboczny.
  - Elektrony i protony są transportowane przez łańcuch transportu elektronów, co prowadzi do wytworzenia ATP i NADPH.

2. Faza ciemna (cykl Calvina, reakcje niezależne od światła):
  - Zachodzi w stromie chloroplastów.
  - ATP i NADPH wytworzone w fazie jasnej są wykorzystywane do przekształcenia dwutlenku węgla (CO

# Przykład 3

Praca na tekście

In [ ]:
temperature = 0.1
max_tokens = 512

In [ ]:
temperature = 0.1
max_tokens = 512
review = "Przy waszych odsetkach to tylko chleb i margaryna!"
prompt = f"Czy podana opinia jest pozytywna czy negatywna i wyjaśnij to obszernie.\n\n###OPINIA:\n{review}"

generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



###ANALIZA:

Ta opinia wydaje się być negatywnna. Oto szczegółowe wyjaśnienie:

1. Kontekst: Opinia ta prawdopodobnie odnosi się do wysokich odsetek, które są postrzegane jako niekorzystne lub zbyt wysokie.

2. Ton: Użycie słów "tylko chleb i margaryna" sugeruje, że oferta jest uboga lub niezadowalająca. Chleb i margaryna to produkty podstawowe, które nie są uważane za luksusowe ani wyrafinowane.

3. Porównanie: Porównanie do chleba i margaryny może być interpretowane jako krytyka, ponieważ sugeruje, że oferta jest minimalistyczna i nie spełnia oczekiwań.

4. Kontekst ekonomiczny: W kontekście finansowym, wysokie odsetki mogą być postrzegane jako kosztowne i niekorzystne dla konsumentów.

5. Emocjonalny wydźwięk: Użycie takiego porównania może wywoływać negatywne emocje, takie jak frustracja czy niezadowolenie.

Podsumowując, opinia ta jest negatywna, ponieważ krytykuje wysokie odsetki i porównuje je do czegoś podstawowego i niezadowalającego, co sugeruje, że oferta nie spełnia oczeki

In [ ]:
system = "Pracujesz jako specjalista od zarządzania jakością w banku i badasz doświadczenia klientów. Analizujesz opinie, wskazujesz pozytywne aspekty, na które zwrócili uwagę klienci oraz obszary do poprawy. Odpowiadasz w języku polskim."

generate(review, system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



To zdanie jest metaforą, która może oznaczać, że oferta banku jest bardzo ograniczona lub nieatrakcyjna. W kontekście bankowym, "chleb i margaryna" może symbolizować podstawowe, nisko oprocentowane produkty, które nie są zbyt korzystne dla klientów.

Aby lepiej zrozumieć tę metaforę, warto rozważyć kilka możliwych interpretacji:

1. Ograniczona oferta: Bank oferuje tylko podstawowe produkty, bez możliwości wyboru bardziej atrakcyjnych opcji.

2. Niskie oprocentowanie: Produkty bankowe mają niskie stopy procentowe, co nie jest satysfakcjonujące dla klientów.

3. Brak innowacji: Bank nie wprowadza nowych, ciekawych produktów, które mogłyby przyciągnąć klientów.

4. Niska jakość usług: Klienci mogą czuć, że usługi bankowe są podstawowe i nie spełniają ich oczekiwań.

5. Niska rentowność: Inwestycje w banku nie przynoszą oczekiwanych zysków.

Aby lepiej zrozumieć, co dokładnie krytykuje klient, warto zadać pytania:

- Czy chodzi o ograniczoną gamę produktów?
- Czy klienci oczekują wyższyc

In [ ]:
temperature = 0.2

review = """Aplikacja działała świetnie i jest już w języku angielskim...​
Jednak od tygodnia zawiesza się, nie można się zalogować, używam tylko blika i akceptuję płatności...​
Kiedy to zostanie rozwiązane?"""

prompt = f"Czy podana opinia jest pozytywna czy negatywna? Uzasadnij odpowiedź, wymień w punktach aspekty pozytywne oraz wymagające poprawy. \n\n###OPINIA:\n{review}"

generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



###ANALIZA:

####Aspekty pozytywne:
1. **Działanie aplikacji**: Użytkownik wskazuje, że aplikacja działała świetnie, co sugeruje, że funkcjonalność i interfejs były dobrze oceniane.
2. **Język angielski**: Wzmianka o tym, że aplikacja jest już w języku angielskim, wskazuje na to, że jest ona dostępna dla szerszego grona odbiorców.

####Aspekty wymagające poprawy:
1. **Stabilność aplikacji**: Użytkownik zgłasza, że od tygodnia aplikacja się zawiesza, co wskazuje na problemy z jej stabilnością i niezawodnością.
2. **Logowanie**: Trudności z logowaniem się do aplikacji są poważnym problemem, ponieważ uniemożliwiają korzystanie z jej funkcji.
3. **Płatności**: Użytkownik korzysta tylko z jednej metody płatności (BLIK) i akceptuje płatności, co może wskazywać na ograniczenia w zakresie obsługiwanych metod płatności.

###Podsumowanie:
Opinia jest mieszana, zawierająca zarówno pozytywne, jak i negatywne aspekty. Pozytywne elementy dotyczą dobrego działania aplikacji oraz dostępności w języku

# Przykład 4
Praca z dokumentami

In [ ]:
import requests
from bs4 import BeautifulSoup


def get_page_content(url='https://hiperautomatyzacja.pl'):
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    plain_text = soup.get_text()
    clean_text = plain_text
    return clean_text

In [ ]:
temperature = 0.1
max_tokens = 512

system = "Dokonaj streszczenia podanego tekstu strony internetowej. Pisz w języku polskim."
prompt = get_page_content("https://hiperautomatyzacja.pl")

generate(prompt, system=system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Streszczenie:

Konferencja "HiperAutomatyzacja 8" to największe w Polsce wydarzenie poświęcone wdrażaniu rozwiązań RPA, LowCode, NoCode, Chatboty, Voiceboty, MLOps i Process Mining w połączeniu z AI. Odbywa się w dniach 10-11 czerwca 2024 roku w Warszawie (rozgrzewka merytoryczna) i online. Uczestnicy będą mieli możliwość zobaczenia 18 case studies z Polski. Konferencja jest bezpłatna i skierowana do osób zainteresowanych automatyzacją i AI w średnich i dużych firmach. Organizatorzy zapewniają dostęp do nagrań z konferencji oraz nie udostępniają maili uczestników. Konferencja jest częścią przedsięwzięcia budowy polskiej społeczności hiperautomatyzacji i AI - Liderzy.AI.


In [ ]:
prompt = get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt, system=system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Streszczenie:

Program konferencji "HiperAutomatyzacja.pl" zawiera szereg prezentacji i paneli dyskusyjnych dotyczących różnych aspektów automatyzacji procesów biznesowych, w tym zastosowania sztucznej inteligencji (AI) i robotyzacji procesów (RPA). Tematyka obejmuje m.in. hiperautomatyzację w logistyce, prototypowanie automatyzacji z wykorzystaniem GenAI, platformy automatyzujące w skali Enterprise, symbiozę RPA i AI, a także podejście do wdrożenia governance nad dużymi modelami językowymi i AI w Power Platform.

W programie znajdują się również sesje dotyczące efektywnego wdrażania rozwiązań RAG w środowiskach korporacyjnych oraz omówienie wersji 2.0 dokumentu Dobrych Praktyk Robotyzacji Procesów wydanego przez Związek Banków Polskich. Na koniec, uczestnicy będą mogli wziąć udział w dyskusji na temat budowy czatbota z wykorzystaniem otwartego polskiego modelu językowego.

Konferencja składa się z dwóch strumieni: menadżerskiego i wdrożeniowego, co pozwala na dostosowanie treści do r

In [ ]:
system = "Wymień warsztaty dotyczące AI oraz prowadzących z imienia, nazwiska z poniższego tekstu strony internetowej."
prompt = system + "\n\nTEKST:" + get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Warsztaty dotyczące AI oraz prowadzący z imienia i nazwiska:

1. Zbigniew Kępiński - Hiperautomatyzacja w logistyce
2. Tomasz Gawron - Prototypowanie automatyzacji procesów z wykorzystaniem GenAI
3. Maciej Zyzak - Platformy Automatyzujące w skali Enterprise
4. Maciej Rudziński - Symbioza gatunków - RPA i AI w PZU
5. Adam Rataj - RPA i AI dwa bratanki
6. Tomasz Łopuchowicz - Podejście do wdrożenia governance nad dużymi modelami językowymi (LLM) i sztuczną inteligencją w Power Platform
7. Krzysztof Karaszewski - Czy agenci AI zastąpią RPA? - Przegląd technologii i narzędzi AI do automatyzacji
8. Paweł Bulowski - Efektywne wdrażanie rozwiązań RAG w środowiskach korporacyjnych
9. Radosław Repczyński - Wersja 2.0 dokumentu Dobrych Praktyk Robotyzacji Procesów wydanego przez Związek Banków Polskich
10. Jan Maria Kowalski - Bielik w praktyce, czyli jak zbudować czatbota korzystając z otwartego polskiego modelu językowego
11. Sebastian Kondracki - Bielik w praktyce, czyli jak zbudować czatbot

In [ ]:
system = """Wymień warsztaty dotyczące AI z poniższego tekstu strony internetowej. Zwróć w postaci JSON według szablonu
[
  {
    'trener':...,
    'temat':...,
    'godzina': ...
    }
    ,
] """

prompt = system + "\n\nTEKST:" + get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Oto wymagany format JSON:

```json
[
 {
   "trener": "Zbigniew Kępiński",
   "temat": "Hiperautomatyzacja w logistyce",
   "godzina": "09:10-10:10"
 },
 {
   "trener": "Tomasz Gawron",
   "temat": "Prototypowanie automatyzacji procesów z wykorzystaniem GenAI",
   "godzina": "10:10-10:50"
 },
 {
   "trener": "Maciej Zyzak",
   "temat": "Platformy Automatyzujące w skali Enterprise",
   "godzina": "10:50-11:30"
 },
 {
   "trener": "Maciej Rudziński",
   "temat": "Symbioza gatunków - RPA i AI w PZU",
   "godzina": "11:30-12:10"
 },
 {
   "trener": "Adam Rataj",
   "temat": "RPA i AI dwa bratanki",
   "godzina": "12:10-12:50"
 },
 {
   "trener": "Tomasz Łopuchowicz, Artur Szylkin",
   "temat": "Podejście do wdrożenia governance nad dużymi modelami językowymi (LLM) i sztuczną inteligencją w Power Platform",
   "godzina": "12:50-13:30"
 },
 {
   "trener": "Krzysztof Karaszewski",
   "temat": "Czy agenci AI zastąpią RPA? - Przegląd technologii i narzędzi AI do automatyzacji",
   "godzina": "1

In [ ]:
max_tokens = 1024
temperature = 0.1

system = "Pracujesz w agencji rekrutacyjnej. Pomagasz dużej firmie z branży finansowej zatrudnić menedżera do spraw AI. Dokonaj oceny podanego kandydata i wypisz jego mocne strony."
prompt = get_page_content("https://hiperautomatyzacja.pl/o-organizatorze")
generate(prompt, system=system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Przepraszam, ale nie widzę żadnego tekstu, który mógłby być oceniony. Aby dokonać oceny kandydata, potrzebowałbym informacji takich jak:

1. Doświadczenie zawodowe
2. Wykształcenie
3. Umiejętności techniczne
4. Doświadczenie w zarządzaniu projektami AI
5. Znajomość branży finansowej
6. Umiejętności miękkie
7. Osiągnięcia i sukcesy

Na podstawie podanych informacji, mogę wymienić mocne strony kandydata:

1. **Doświadczenie w zarządzaniu projektami**: Kandydat ma ponad 20 lat doświadczenia w analizie biznesowo-systemowej, architekturze korporacyjnej i IT. Pracował w wielu różnych firmach, co świadczy o jego zdolnościach adaptacyjnych i umiejętności zarządzania projektami w różnych środowiskach.

2. **Umiejętności techniczne**: Kandydat ma szeroką wiedzę na temat architektury korporacyjnej, robotyzacji procesów biznesowych (RPA), LowCode i NoCode, oraz sztucznej inteligencji (AI). Jest autorem książek na te tematy, co wskazuje na głęboką wiedzę teoretyczną i praktyczną.

3. **Znajomość b